In [1]:
#rodar código abaixo caso o notebook não reconheça o ambiente virtual
#!python -m ipykernel install --user --name=nome_da_pasta_com_o_ambiente_virtual

In [2]:
from pathlib import Path
import duckdb
import pandas as pd

In [3]:
conn = duckdb.connect(Path('../data/db_games_sales.db'))

In [4]:
conn.sql("DESCRIBE games_sales").df()

,column_name,column_type,null,key,default,extra
0,index,BIGINT,YES,None,None,None
1,Name,VARCHAR,YES,None,None,None
2,Platform,VARCHAR,YES,None,None,None
3,Year,BIGINT,YES,None,None,None
4,Genre,VARCHAR,YES,None,None,None
5,Sales_NA,DOUBLE,YES,None,None,None
6,Sales_EU,DOUBLE,YES,None,None,None
7,Sales_JP,DOUBLE,YES,None,None,None
8,Sales_Others,DOUBLE,YES,None,None,None
9,Global_Sales,DOUBLE,YES,None,None,None


In [5]:
conn.sql("""
SELECT *
FROM games_sales
LIMIT 10
""").df()

,index,Name,Platform,Year,Genre,Sales_NA,Sales_EU,Sales_JP,Sales_Others,Global_Sales
0,0,'98 Koshien,PS,1998,Sports,0.15,0.10,0.12,0.03,0.40
1,1,.hack//G.U. Vol.1//Rebirth,PS2,2006,Role-Playing,0.00,0.00,0.17,0.00,0.17
2,2,.hack//G.U. Vol.2//Reminisce,PS2,2006,Role-Playing,0.11,0.09,0.00,0.03,0.23
3,3,.hack//G.U. Vol.2//Reminisce (jp sales),PS2,2006,Role-Playing,0.00,0.00,0.16,0.00,0.16
4,4,.hack//G.U. Vol.3//Redemption,PS2,2007,Role-Playing,0.00,0.00,0.17,0.00,0.17
5,5,.hack//Infection Part 1,PS2,2002,Role-Playing,0.49,0.38,0.26,0.13,1.26
6,6,.hack//Link,PSP,2010,Role-Playing,0.00,0.00,0.14,0.00,0.14
7,7,.hack//Mutation Part 2,PS2,2002,Role-Playing,0.23,0.18,0.20,0.06,0.67
8,8,.hack//Outbreak Part 3,PS2,2002,Role-Playing,0.14,0.11,0.17,0.04,0.46
9,9,.hack//Quarantine Part 4: The Final Chapter,PS2,2003,Role-Playing,0.09,0.07,0.00,0.02,0.18


# Análise

### Quais foram os TOP 3 jogos mais vendidos de 2015?

In [6]:
conn.sql("""
SELECT Name,
       Global_Sales
FROM games_sales
WHERE YEAR = 2015
ORDER BY Global_Sales DESC
LIMIT 3
""").df()

,Name,Global_Sales
0,Call of Duty: Black Ops 3,14.24
1,FIFA 16,8.49
2,Star Wars Battlefront (2015),7.67


### Qual foi a média de jogos vendidos pelo XOne no ano de 2016?

In [7]:
#procura o nome da plataforma que corresponde ao console Xbox One
conn.sql("""
SELECT DISTINCT Platform
FROM games_sales
WHERE Platform LIKE '%X%'
""").df()

,Platform
0,X360
1,XB
2,XOne
3,PCFX


In [8]:
conn.sql("""
    SELECT 
        MEAN(Global_Sales) as Média
    FROM 
        games_sales
    WHERE 
        Platform = 'XOne' AND Year = 2016
""").df().map(lambda x: '{:,.0f}'.format(x * 1000000))#formata os valores do df para a casa de milhares


,Média
0,"229,259"


### Qual a representatividade do gênero de Sports frente às demais a partir do ano 2000?

In [9]:
conn.sql("""
SELECT CONCAT(ROUND((SUM(CASE
                             WHEN Genre = 'Sports' THEN Global_Sales
                             ELSE 0
                         END) * 100.0) / SUM(Global_Sales), 2), '%') AS Sports
FROM games_sales
WHERE YEAR >= 2000
""").df()

,Sports
0,15.78%


### Qual foi o jogo mais vendido no Japão em 1998?

In [10]:
conn.sql("""
SELECT Name as 'Top 1',
       Sales_JP
FROM games_sales
WHERE YEAR = 1998
ORDER BY Sales_JP DESC
LIMIT 1
""").df()

,Top 1,Sales_JP
0,Pokémon Yellow: Special Pikachu Edition,3.12


### Quais são as porcentagens de vendas por região para cada um dos 25 jogos com as maiores vendas globais?

In [11]:
conn.sql("""
SELECT Name,
       Global_Sales,
       CONCAT(Sales_NA, '(', ROUND(Sales_NA / Global_Sales * 100, 1), '%)') AS NA,
       CONCAT(Sales_EU, '(', ROUND(Sales_EU / Global_Sales * 100, 1), '%)') AS EU,
       CONCAT(Sales_JP, '(', ROUND(Sales_JP / Global_Sales * 100, 1), '%)') AS JP,
       CONCAT(Sales_Others, '(', ROUND(Sales_Others / Global_Sales * 100, 1), '%)') AS Others
FROM games_sales
ORDER BY Global_Sales DESC
LIMIT 25
""").df()

,Name,Global_Sales,NA,EU,JP,Others
0,Wii Sports,82.74,41.49(50.1%),29.02(35.1%),3.77(4.6%),8.46(10.2%)
1,Super Mario Bros.,40.24,29.08(72.3%),3.58(8.9%),6.81(16.9%),0.77(1.9%)
2,Mario Kart Wii,35.83,15.85(44.2%),12.88(35.9%),3.79(10.6%),3.31(9.2%)
3,Wii Sports Resort,33.00,15.75(47.7%),11.01(33.4%),3.28(9.9%),2.96(9.0%)
4,Pokemon Red/Pokemon Blue,31.38,11.27(35.9%),8.89(28.3%),10.22(32.6%),1.0(3.2%)
5,Tetris,30.26,23.2(76.7%),2.26(7.5%),4.22(13.9%),0.58(1.9%)
6,New Super Mario Bros.,30.01,11.38(37.9%),9.23(30.8%),6.5(21.7%),2.9(9.7%)
7,Wii Play,29.01,14.03(48.4%),9.2(31.7%),2.93(10.1%),2.85(9.8%)
8,New Super Mario Bros. Wii,28.61,14.59(51.0%),7.06(24.7%),4.7(16.4%),2.26(7.9%)
9,Duck Hunt,28.31,26.93(95.1%),0.63(2.2%),0.28(1.0%),0.47(1.7%)
